# Module 05: Linear Regression with Nonlinear Effects

In [ ]:
# packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)

# set seed
seed = 4721

### We'll use the _Bikeshare_ data from ISLP for this activity. The metadata for _Bikeshare_ can be found [here](https://intro-stat-learning.github.io/ISLP/datasets/Bkeshare.html).

In [ ]:
# Load the data
Bikeshare = load_data('Bikeshare')

In [ ]:
# List the columns and their types
Bikeshare.dtypes

### We will predict the `bikers` column, which is the total of casual and registered bikers. 
Then we can create a matrix of potential independent variables (X) and a dependent variable vector (y).

In [ ]:
indep_vars = ['season', 'mnth', 'day', 'hr', 
              'holiday', 'weekday', 'workingday', 
              'weathersit', 'temp', 'atemp', 'hum', 'windspeed']

X = Bikeshare[indep_vars]
y = Bikeshare['bikers']

### Before doing any other analyses, let's create training and test sets.

In [ ]:
X_train, X_test, y_train, y_test, Train, Test = train_test_split(X, y, Bikeshare, 
                                                                 random_state = seed, 
                                                                 test_size = 0.25, 
                                                                 shuffle = True)
                             

### We can first summarize the variables in the training data.

In [ ]:
Train.describe()

### Then we can create plots for each potential independent variable with `bikers` on training data.

In [ ]:
# Initialize the plots before drawing them
nrows = 4
ncols = 3
figsize = (5*nrows, 5*ncols)

fig, axes = subplots(nrows=nrows,
                     ncols=ncols,
                     figsize=figsize)

# Assign a grid location to each index
def range_to_grid(i, nrows, ncols):
    x=[]
    y=[]
    for n in range(nrows*ncols):
        x.append(n // ncols)
        y.append(n % ncols)
        # print(n,x[n],y[n]) # for testing this function
    return x[i],y[i]

# Plot the variables
for j in range(len(X_train.columns)):
    # print(range_to_grid(j,nrows,ncols)[0], range_to_grid(j,nrows,ncols)[1]) # testing
    axes[range_to_grid(j,nrows,ncols)[0],
         range_to_grid(j,nrows,ncols)[1]].plot(X_train.iloc[:,j], y_train, 'o')
    axes[range_to_grid(j,nrows,ncols)[0],
         range_to_grid(j,nrows,ncols)[1]].set_xlabel(X_train.columns[j])

### Since we see that the `hr` variable is periodic (with period 24), we will use trigonometric transforms as our spline basis functions

In [ ]:
# Data preprocessing
Train['hr_sin'] = round(np.sin(2*np.pi/24*Train['hr'].astype('int')),2)
Test['hr_sin'] = round(np.sin(2*np.pi/24*Test['hr'].astype('int')),2)
X_train['hr_sin'] = round(np.sin(2*np.pi/24*X_train['hr'].astype('int')),2)
X_test['hr_sin'] = round(np.sin(2*np.pi/24*X_test['hr'].astype('int')),2)

Train['hr_cos'] = round(np.cos(2*np.pi/24*Train['hr'].astype('int')),2)
Test['hr_cos'] = round(np.cos(2*np.pi/24*Test['hr'].astype('int')),2)
X_train['hr_cos'] = round(np.cos(2*np.pi/24*X_train['hr'].astype('int')),2)
X_test['hr_cos'] = round(np.cos(2*np.pi/24*X_test['hr'].astype('int')),2)

# spot-check this
Train.head()

## First Linear Regression Model with a spline

### We can create a column to represent the intercept in our model. 

Some packages do this automatically, but we will leverage the ISLP version, which does not.

In [ ]:
X_train['intercept'] = np.ones(X_train.shape[0])
X_test['intercept'] = np.ones(X_test.shape[0])

### We'll build a regression model to predict `bikers` that uses a straight line for `temp` and a spline with trig basis functions for `hr`.

In [ ]:
model_4 = sm.OLS(y_train, X_train[['intercept', 'temp', 'hr_sin', 'hr_cos']])
results_4 = model_4.fit()
summarize(results_4)

### We will evaluate this model *three ways*: Using R^2, using MSE, and then visually. 

For the last two, we will compare on _Train_ and on _Test_.

In [ ]:
# R^2 on training
results_4.rsquared

In [ ]:
# Create helper functions for computing the mean squared error

def predict(X, model):
    # the built-in get_prediction tool returns an array, so we need to convert to a dataframe
    predictions_df = pd.DataFrame(model.get_prediction(X).predicted, columns=['y_hat'], index=X.index)
    return predictions_df['y_hat']

def mse(y, y_hat):
    # calculate the residual error for each individual record
    resid = y - y_hat
    # square the residual (hence "squared error")
    sq_resid = resid**2
    # calculate the sum of squared errors
    SSR = sum(sq_resid)
    # divide by the number of records to get the mean squared error
    MSE = SSR / y.shape[0]
    return MSE

In [ ]:
predictions_train_4 = predict(X_train[['intercept', 'temp', 'hr_sin', 'hr_cos']], results_4)
print('MSE train: ', mse(y_train, predictions_train_4))

predictions_test_4 = predict(X_test[['intercept', 'temp', 'hr_sin', 'hr_cos']], results_4)
print('MSE test: ', mse(y_test, predictions_test_4))

### Here is a helpful tool for evaluating our model(s). 

Since the variables we're using aren't always being fit with straight lines, it's helpful to plot the average _actual_ y-value to the average _predicted_ y-value for each modeled feature.

**Note**: It is **not** recommended to leverage this tool on test data. This can quickly lead to unintentional overfitting on the test set, which could lead to a poor model on new data.

In [ ]:
def plot_avg_actual_vs_predicted_by_feature(X, y, y_pred, feature, target_name="Target"):
    df = pd.DataFrame({
        feature: X[feature],
        "actual": y,
        "predicted": y_pred
    })

    grouped = df.groupby(feature).mean().reset_index()

    plt.figure()
    plt.plot(grouped[feature], grouped["actual"], label="Average Actual")
    plt.plot(grouped[feature], grouped["predicted"], label="Average Predicted")
    plt.xlabel(feature)
    plt.ylabel(target_name)
    plt.legend()
    plt.show()

In [ ]:
plot_avg_actual_vs_predicted_by_feature(X=X_train, 
                                        y=y_train, 
                                        y_pred=predictions_train_4, 
                                        feature='temp', 
                                        target_name="Bikers"
                                        )

In [ ]:
plot_avg_actual_vs_predicted_by_feature(X=X_train, 
                                        y=y_train, 
                                        y_pred=predictions_train_4, 
                                        feature='hr', 
                                        target_name="Bikers"
                                        )

### We are seeing that there are spikes in bike rentals at rushhour times: 7am, 8am, 5pm, and 6pm. Let's incorporate a categorical variable to account for these. 

In [ ]:
# Data preprocessing
Train["rushhour"] = np.where(Train["hr"].astype('int').isin([7, 8, 17, 18]), 1, 0)
Test["rushhour"] = np.where(Test["hr"].astype('int').isin([7, 8, 17, 18]), 1, 0)
X_train["rushhour"] = np.where(X_train["hr"].astype('int').isin([7, 8, 17, 18]), 1, 0)
X_test["rushhour"] = np.where(X_test["hr"].astype('int').isin([7, 8, 17, 18]), 1, 0)

# spot-check this
Train[['hr', "rushhour"]].head(12)

In [ ]:
model_5 = sm.OLS(y_train, X_train[['intercept','temp','hr_sin', 'hr_cos', 'rushhour']])
results_5 = model_5.fit()
summarize(results_5)

### Compute the R^2 coefficient for the new model.

In [ ]:
results_5.rsquared

### Compute the MSE on the training and test sets for the new model. 

In [ ]:
predictions_train_5 = predict(X_train[['intercept', 'temp', 'hr_sin', 'hr_cos', 'rushhour']], results_5)
print('MSE train: ', mse(y_train, predictions_train_5))

predictions_test_5 = predict(X_test[['intercept', 'temp', 'hr_sin', 'hr_cos', 'rushhour']], results_5)
print('MSE test: ', mse(y_test, predictions_test_5))

In [ ]:
plot_avg_actual_vs_predicted_by_feature(X=X_train, 
                                        y=y_train, 
                                        y_pred=predictions_train_5, 
                                        feature='temp', 
                                        target_name="Bikers"
                                        )

In [ ]:
plot_avg_actual_vs_predicted_by_feature(X=X_train, 
                                        y=y_train, 
                                        y_pred=predictions_train_5, 
                                        feature='hr', 
                                        target_name="Bikers"
                                        )

# Now, for your goal:

### Use the tools we've talked about in this module (categorical variables, interactions, splines, etc.) to build the best possible linear regression model. 

## **DO NOT** do any of the following, which would result in an invalid submission:
- Use techniques other than linear regression (specifically, you must use sm.OLS)
- Use postdictors (bikers, casual, registered) to predict bikers
- Change the random seed or anything with the train/test split
- Change any of the variables defined below this cell (model_final, results_final, predictions_train_final, predictions_test_final)
- Manually manipulate predictions

In [ ]:
# Data pre-processing (on Train, Test, X_train, and X_test)


In [ ]:
# Build your linear regression model here.
vars_to_include = [
    #fillin
] 

model_final = sm.OLS(y_train, X_train[vars_to_include])
results_final = model_final.fit()
summarize(results_final)

In [ ]:
predictions_train_final = predict(X_train[vars_to_include], results_final)
print('MSE train: ', mse(y_train, predictions_train_final))

predictions_test_final = predict(X_test[vars_to_include], results_final)
print('MSE test: ', mse(y_test, predictions_test_final))

In [ ]:
plot_avg_actual_vs_predicted_by_feature(X=X_train, 
                                        y=y_train, 
                                        y_pred=predictions_train_final, 
                                        feature='temp', 
                                        target_name="Bikers"
                                        )